# Assessment for Computer Infrastructure

## Problem 1: 
### Using the yfinance Python package, write a function called get_data() that downloads all hourly data for the previous five days for the five FAANG stocks.

### As a first step, I created a folder called 'data' in my repository using the mkdir command in the terminal. See: https://how.dev/answers/how-to-add-a-folder-to-a-repository-in-github 

In [ ]:
# Yahoo Finance data.
import yfinance as yf


In [ ]:
# https://www.geeksforgeeks.org/python/python-os-path-join-method/ 
import os

In [2]:
# for naming a file using a date or time, see: https://www.geeksforgeeks.org/python/how-to-create-filename-containing-date-or-time-in-python/
from datetime import datetime 

In [3]:
# practice getting the right filename - delete this later on
current_datetime = datetime.now().strftime("%Y%m%d-%H%M%S")
print(current_datetime)

20251014-201416


In [12]:
# create a function as per https://www.geeksforgeeks.org/python/python-functions/ 
def get_data():
    # Access multiple tickers as per documentation (https://ranaroussi.github.io/yfinance/reference/index.html).
    # adapted code on importing historical data from here: https://rowzero.io/blog/yfinance 
    # specific valid intervals found here: https://medium.com/@anjalivemuri97/day-4-fetching-historical-stock-data-with-yfinance-f45f3bd8b9c6 
    data = yf.download("META AAPL AMZN NFLX GOOG", group_by='tickers', period="5d", interval='1h')

    #create the time stamp that can dynamically name the file. See: https://www.geeksforgeeks.org/python/how-to-create-filename-containing-date-or-time-in-python/
    current_datetime = datetime.now().strftime("%Y%m%d-%H%M%S")

    # convert the output into a string
    current_datetime_str = str(current_datetime)

    # name the file with .csv extension
    file_name = current_datetime_str+".csv"

    # create a folder - importantly - in the current working directory, not just in your codespace area - see: https://www.w3schools.com/python/ref_os_getcwd.asp 
    dir = os.path.join(os.getcwd(), "data")

    # create a path using os path join which concatenates the file path and the filename and saves it to file_path (see: https://blog.finxter.com/how-to-save-a-text-file-to-another-folder-in-python/)
    file_path = os.path.join(dir, file_name)
    
    data.to_csv(file_path)
    

In [5]:
# print the data
#print(data)